In [ ]:
import os
import pandas as pd
from ebmdatalab import bq

In [7]:
sql = f"""
SELECT 
CAST (month AS date) AS month, -- Converts TIMESTAMP to DATE (as it's always just 1st of month)
regional_team,
bnf_code,
bnf_name,
CASE -- This uses the BNF hierarchy to classify by drug
  WHEN bnf_code LIKE '0601023AQ%' THEN 'Dulaglutide' 
  WHEN bnf_code LIKE '0601023Y0%' THEN 'Exenatide'
  WHEN bnf_code LIKE '0601023AB%' THEN 'Liraglutide'
  WHEN bnf_code LIKE '0601023AI%' THEN 'Lixisenatide'
  WHEN bnf_code LIKE '0601023AW%' THEN 'Semaglutide'
  WHEN bnf_code LIKE '0601023AZ%' THEN 'Tirzepatide'
  WHEN bnf_code LIKE '0601023AS%' THEN 'Albiglutide'
  WHEN bnf_code LIKE '0601023AU%' THEN 'Insulin degludec/liraglutide'
  WHEN bnf_code LIKE '0601012AB%' THEN 'Insulin glargine/lixisenatide'
  -- Add others here as needed
END AS drug,
SUM(items) AS items,
SUM(quantity) AS quantity,

FROM `ebmdatalab.hscic.normalised_prescribing` np
LEFT JOIN ebmdatalab.hscic.practices ON np.practice = practices.code -- We bring this table in so we can filter to GP practices only (see setting below)

WHERE
month >= '2020-01-01'
AND
(
  bnf_code LIKE '0601023AQ%' -- Dulaglutide
  OR bnf_code LIKE '0601023Y0%' -- Exenatide
  OR bnf_code LIKE '0601023AB%' -- Liraglutide
  OR bnf_code LIKE '0601023AI%' -- Lixisenatide
  OR bnf_code LIKE '0601023AW%' -- Semaglutide
  OR bnf_code LIKE '0601023AZ%' -- Tirzepatide
  OR bnf_code LIKE '0601023AS%' -- Albiglutide -- appears to have been discontinued
  -- Below are GLP-1's in combination with insulin
  OR bnf_code LIKE '0601023AU%' -- Insulin degludec/liraglutide
  OR bnf_code LIKE '0601012AB%' -- Insulin glargine/lixisenatide
)
AND practices.setting = 4 -- This restricts to GP practices only
GROUP BY -- Needed as we are selecting SUM of items and quantity
month,
regional_team,
bnf_code,
bnf_name
"""

# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'vtm_matched.csv')

# Use the cached_read function from the bq library to run the query.
vtm_match = bq.cached_read(sql, csv_path=csv_path)

Downloading: 100%|█████████████████████████████████████████████████████████████|
